## Notes

### TODO
- Enhance feedback controller to handle sensor wrap more robustly
 - Potentially force an initial sensor offset of 180 (to maximize operating space before wrap)
 - Probably need to enforce a max turn rate to avoid ambiguity between wakeups
- Potentially switch controller from mixed mode to differential mode
- Add mutexes and condition variables in several places to increase robustness

### Overview


### READ THIS BEFORE RUNNING
- Some of the following experiments depend on the PIGPIO daemon.  To activate the daemon process, execute the following command on the host machine for this notebook: "sudo pigpiod" (Note: This should be done before running the experiment cells that depend on it.)

### How to run this notebook
##### Launch jupyter notebook server on RPi
- From a local terminal session, log into the RPi on the rover
- run "source ~/.profile" to make sure virtualenvwrapper environment variables are set
- run "workon rover1" to launch the correct virtual environment
- cd to the repo root directory (nominally ~/nanibot)
- run "jupyter notebook --no-browser --port=[port #]" to launch the notebook server

##### Setup ssh tunneling and port forwarding on local machine
- run "ssh -N -f -L localhost:[port #]:localhost:[port #] pi@[RPi host IP]"
- From local browser, go to localhost:[port #]

### Implementation Details

### Test Design
 
### Conclusions

### Next Steps


## Component Code

In [11]:
%matplotlib inline
from __future__ import division
import time
import sys
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from Adafruit_BNO055 import BNO055
import serial
from serial.serialutil import SerialException
import multiprocessing as multiproc

In [12]:
# -- Define some configuration constants for the imu sensor --
CALIBRATION_FILE='/home/pi/nanibot/rover1/peripherals/bno055_imu/calibration/calibration.json'
CALIBRATION_DATA=[1, 0, 2, 0, 244, 255, 223, 255, 50, 1, 210, 255, 253, 255, 255, 255, 2, 0, 232, 3, 181, 2]

BNO_AXIS_REMAP = {
        'x': BNO055.AXIS_REMAP_Y,
        'y': BNO055.AXIS_REMAP_X,
        'z': BNO055.AXIS_REMAP_Z,
        'x_sign': BNO055.AXIS_REMAP_NEGATIVE,
        'y_sign': BNO055.AXIS_REMAP_POSITIVE,
        'z_sign': BNO055.AXIS_REMAP_POSITIVE }

In [13]:
# - Add the main rover source tree to sys.path so modules are accessible - 
rover1_path = '/home/pi/nanibot/rover1'
if rover1_path not in sys.path:
    sys.path.insert(0,rover1_path)
sys.path

['/home/pi/nanibot/rover1',
 '',
 '/usr/local/lib/python2.7/dist-packages/Adafruit_BNO055-1.0.1-py2.7.egg',
 '/usr/local/lib/python2.7/dist-packages/Adafruit_GPIO-1.0.0-py2.7.egg',
 '/usr/local/lib/python2.7/dist-packages/Adafruit_PureIO-0.2.0-py2.7.egg',
 '/usr/lib/python2.7/dist-packages',
 '/usr/lib/python2.7',
 '/usr/lib/python2.7/plat-arm-linux-gnueabihf',
 '/usr/lib/python2.7/lib-tk',
 '/usr/lib/python2.7/lib-old',
 '/usr/lib/python2.7/lib-dynload',
 '/usr/local/lib/python2.7/dist-packages',
 '/usr/lib/python2.7/dist-packages/gtk-2.0',
 '/usr/lib/pymodules/python2.7',
 '/usr/local/lib/python2.7/dist-packages/IPython/extensions',
 '/home/pi/.ipython']

In [14]:
import peripherals.bno055_imu.inertial_sensor_bno055 as imu
import peripherals.sabertooth.sabertooth_adapter as sabertooth_adapter
import components.driving.motor_control as motor_control

In [58]:
reload(imu)

<module 'peripherals.bno055_imu.inertial_sensor_bno055' from '/home/pi/nanibot/rover1/peripherals/bno055_imu/inertial_sensor_bno055.pyc'>

In [15]:
class HeadingEstimator(object):
    
    def __init__(self,sensor):
        
        self._sensor = sensor
        self._last_validity_time = time.time()
        self._last_heading = 0
        
    @property
    def last_heading(self):
        return self._last_heading
    
    @property
    def last_validity_time(self):
        return self._last_validity_time
    
    def _updateStateEstimates(self):
        
        meas = self._sensor.get_measurement()
        # - Update state estimates
        self._last_validity_time = meas['update_time']
        self._last_heading = meas['heading']
        
    def getCurrentState(self):
        self._updateStateEstimates()

        return {
            'heading':self.last_heading,
            'validity_time':self.last_validity_time}

In [91]:
class FeedbackControlSystemManager(object):
    
    def __init__(self):
        self._control_loop_stop_flag = multiproc.Value('b',0)
        self._set_point = multiproc.Value('f',0.0)
        
    @property
    def control_loop_stop_flag(self):
        return self._control_loop_stop_flag
    
    @property
    def set_point(self):
        return self._set_point
    
    def update_set_point(self,new_set_point):
        self._set_point.value = new_set_point
    
    def stop_control_loop(self):
        self._control_loop_stop_flag.value = 1
        self._feedback_iteration_proc.join(10)
    
    def launch_heading_feedback_control_system(self):
        
        # Make sure stop flag is initialized
        self._clear_control_loop_stop_flag()
        
        self._feedback_iteration_proc = multiproc.Process(
            target=self._heading_feedback_control_loop,
            args=(
                self.control_loop_stop_flag, # stop_control_loop
                self.set_point, # heading_set_point
                0.1)) # update_interval_sec
        #self._feedback_iteration_proc.daemon = True  # Don't let the BNO reading thread block exiting.
        self._feedback_iteration_proc.start()
        
    def _clear_control_loop_stop_flag(self):
        self._control_loop_stop_flag.value = 0
    
    def _heading_feedback_control_loop(
        self,
        stop_control_loop,
        heading_set_point,
        update_interval_sec):
        
        sensor_update_frequency_hz = 10.0
        
        # - Construct Feedback Control System Components
        motorControllerAdapter = sabertooth_adapter.SabertoothPacketizedAdapterGPIO()
        motor_controller = motor_control.MotorController(motorControllerAdapter)
        heading_sensor = imu.MultiprocessHeadingSensorBNO055(
            sensor_update_frequency_hz=sensor_update_frequency_hz)
        heading_estimator = HeadingEstimator(heading_sensor)

        feedback_controller = HeadingFeedbackController(
            heading_estimator, # observer
            motor_controller, # motor_controller
            update_interval_sec, # update_interval_sec
            0.4, #proportional_gain
            0.0, # integral_gain
            measurement_offset=0,
            initial_state=heading_set_point.value,
            nominal_forward_power=25,
            verbose=False)
        
        while stop_control_loop.value == 0:
            # TODO: add mutex around this
            feedback_controller.update_set_point(heading_set_point.value)
            feedback_controller.update_plant_command()
            time.sleep(update_interval_sec)
        
        motor_controller.stop()
        raise ValueError('Stopped updating feedback controller because of stop condition.')
        
        

class HeadingFeedbackController(object):
    
    _observer_timeout_sec = 5
    
    def __init__(
        self,
        observer,
        motor_controller,
        update_interval_sec,
        proportional_gain,
        integral_gain,
        measurement_offset=0,
        initial_state=0,
        nominal_forward_power=0,
        verbose=False):
        
        self._verbose = verbose
        self._break_feedback_iterator = False
        
        # Set the interval on which the feedback controller will try to make updates
        self._update_interval_sec = update_interval_sec
        # Set heading observer
        self._observer = observer
        # Set motor controller and ensure that we're initially not driving
        self._motor_controller = motor_controller
        self._nominal_forward_power = nominal_forward_power
        self._motor_controller.stop()
        self._driving = False
        
        # Set the feedback gains
        self._P_gain = proportional_gain
        self._I_gain = integral_gain
        
        # Set the measurement calibration offset
        self._measurement_offset = measurement_offset
        
        # Initialize last observer time as current time
        self._last_observer_update_time = time.time()
        # Initialize last controller update time as current time
        self._last_controller_update_time = time.time()
        # Keep track of initial invocation time, for logging and debugging
        self._start_time = time.time()
        
        self._set_point = initial_state
        self._last_heading_estimate = initial_state
        self._last_plant_command = initial_state
        self._cumulative_error = 0
        
        
    @property
    def set_point(self):
        return self._set_point
    
    def update_set_point(self,set_point):
        """TODO: add safety limits"""
        self._set_point = set_point
        
    def update_plant_command(self):
        """
        Attempt to update the motor controller command, send stop command if something goes wrong
        
        """
        try:
            # If we're not yet driving, send nominal forward command to motor controller
            if not self._driving:
                # Set nominal forward power on motor controller
                self._motor_controller.goForward(power_percent=self._nominal_forward_power)
                self._driving = True
            
            new_plant_command = self._get_new_plant_command()
            old_plant_command = self._motor_controller.currentLeftRightSetting
            heading_change = new_plant_command - old_plant_command
            self._motor_controller.adjustLeftRightSetting(heading_change)
        except:
            self._motor_controller.stop()
            raise
        
    def _get_control_error(self,measurement,set_point):
        measurement -= 180
        set_point -= 180
        diff = set_point - measurement
        if diff > 180:
            return diff - 360
        elif diff < -180:
            return 360 + diff
        else:
            return diff
        
    def _get_new_plant_command(self):
        # Compute new motor command from proportional feedback
        observer_data = self._observer.getCurrentState()
        state_validity_time = observer_data['validity_time']
        
        # If observer data is too old, raise exception that should shut down the motors
        current_time = time.time()
        if current_time - state_validity_time >= self._observer_timeout_sec:
            error_str_details = "{0:.1f} (current) - {1:.1f} (validity) = {2:.1f} (diff)".format(
                current_time,
                state_validity_time,
                current_time-state_validity_time)
            raise ValueError('observer data has become too stale.\n\t'+error_str_details)
            
        # Make sure there's a valid set point and heading measurement 
        # - Should raise ValueError otherwise
        gain = float(self._P_gain)
        set_point = float(self.set_point)
        calibrated_state_estimate = float(observer_data['heading']) - float(self._measurement_offset)
        
        # --- Proportional Error Feedback ---
        new_plant_command = gain * self._get_control_error(calibrated_state_estimate,set_point)
        if self._verbose:
            print '\n\ntime: {}\nset point: {}\ncalibrated observer heading: {}, \nnew command: {}'.format(
                current_time - self._start_time,
                set_point,
                calibrated_state_estimate,
                new_plant_command)
            
        return new_plant_command
    

### Feedback System Component Construction

In [17]:
sensor_update_frequency_hz = 10.0

motorControllerAdapter = sabertooth_adapter.SabertoothPacketizedAdapterGPIO()
motor_controller = motor_control.MotorController(motorControllerAdapter)
heading_sensor = imu.MultiprocessHeadingSensorBNO055(
    sensor_update_frequency_hz=sensor_update_frequency_hz)
heading_estimator = HeadingEstimator(heading_sensor)

motor command:  131   2   10   15


### Heading Sensor Commands

In [18]:
heading_sensor.get_measurement()

{'heading': 359.9375,
 'lookup_time': 1485908657.197098,
 'update_count': 32.0,
 'update_time': 1485908657.158307}

In [120]:
heading_sensor.shutdown()

In [109]:
heading_estimator.getCurrentState()

{'update_time': 1485746305.249094, 'heading': 47.75, 'lookup_time': 1485746305.353655, 'update_count': 15290.0}


{'heading': 47.75, 'validity_time': 1485746305.249094}

### Motor Controller Commands 

In [24]:
motor_controller.goStraight()

motor command:  131   13   64   80


In [25]:
motor_controller.adjustFwdBwdSetting(power_change=20)

motor command:  131   12   102   117


In [51]:
motor_controller.stop()

motor command:  131   0   0   3
motor command:  131   4   0   7


## New Feedback Control System 

In [92]:
control_system = FeedbackControlSystemManager()

In [138]:
control_system.stop_control_loop()

In [137]:
control_system.update_set_point(180)

In [121]:
control_system.launch_heading_feedback_control_system()

motor command:  131   2   10   15
motor command:  131   0   0   3
motor command:  131   4   0   7
motor command:  131   12   80   95
{'update_time': 1485912488.090583, 'heading': 359.9375, 'lookup_time': 1485912488.134451, 'update_count': 2.0}
motor command:  131   13   65   81
{'update_time': 1485912488.302496, 'heading': 359.9375, 'lookup_time': 1485912488.357386, 'update_count': 4.0}
motor command:  131   13   65   81
{'update_time': 1485912488.514857, 'heading': 0.125, 'lookup_time': 1485912488.59004, 'update_count': 6.0}
motor command:  131   13   62   78
{'update_time': 1485912488.863182, 'heading': 0.25, 'lookup_time': 1485912488.874756, 'update_count': 9.0}
motor command:  131   13   62   78
{'update_time': 1485912489.070939, 'heading': 359.9375, 'lookup_time': 1485912489.09432, 'update_count': 11.0}
motor command:  131   13   65   81
{'update_time': 1485912489.281067, 'heading': 359.8125, 'lookup_time': 1485912489.312762, 'update_count': 13.0}
motor command:  131   13   65   8

## Sandbox

## Experiments
(Continuation from http://localhost:1234/notebooks/sandboxes/ipython_notebooks/imu_multiproc_sampling_and_buffering_test_2.ipynb )

In [7]:
# - Plotting helper for several experiments
def plot_raw_and_processed_data(
    raw_data,
    filtered_data=None,
    y1lim=None,
    y2lim=None,
    y3lim=None):
    
    do_filtered_data = False
    if filtered_data is not None:
        do_filtered_data = True
    

    fig, axes = plt.subplots(figsize=(8,10),nrows=3, ncols=1)

    current_axes = axes[0]
    current_axes.plot(raw_data.x, linestyle='-', color='b', linewidth=4, alpha=0.5, label='raw')
    if do_filtered_data:
        current_axes.plot(filtered_data.x, linestyle='-', color='k', linewidth=1, alpha=1.0, label='filtered')
    current_axes.set_title('Linear Acceleration - X', fontsize=10)
    current_axes.set_ylabel('Signals', fontsize=10)
    if y1lim is not None:
        current_axes.set_ylim(y1lim)
    current_axes.legend(loc='upper left')
    current_axes.grid(which='major',axis='both')

    current_axes = axes[1]
    current_axes.plot(raw_data.y, linestyle='-', color='b', linewidth=4, alpha=0.5, label='raw')
    if do_filtered_data:
        current_axes.plot(filtered_data.y, linestyle='-', color='k', linewidth=1, alpha=1.0, label='filtered')
    current_axes.set_title('Linear Acceleration - Y', fontsize=10)
    current_axes.set_ylabel('Signals', fontsize=10)
    if y2lim is not None:
        current_axes.set_ylim(y2lim)
    current_axes.legend(loc='upper left')
    current_axes.grid(which='major',axis='both')

    current_axes = axes[2]
    current_axes.plot(raw_data.z, linestyle='-', color='b', linewidth=4, alpha=0.5, label='raw')
    if do_filtered_data:
        current_axes.plot(filtered_data.z, linestyle='-', color='k', linewidth=1, alpha=1.0, label='filtered')
    current_axes.set_title('Linear Acceleration - Z', fontsize=10)
    current_axes.set_ylabel('Signals', fontsize=10)
    if y3lim is not None:
        current_axes.set_ylim(y3lim)
    current_axes.legend(loc='upper left')
    current_axes.grid(which='major',axis='both')
    
# - Plotting helper for several experiments
def plot_euler_data(
    raw_data,
    filtered_data=None,
    y1lim=None,
    y2lim=None,
    y3lim=None):
    
    do_filtered_data = False
    if filtered_data is not None:
        do_filtered_data = True

    fig, axes = plt.subplots(figsize=(8,10),nrows=3, ncols=1)

    current_axes = axes[0]
    current_axes.plot(raw_data.heading, linestyle='-', color='b', linewidth=4, alpha=0.5, label='raw')
    if do_filtered_data:
        current_axes.plot(filtered_data.heading, linestyle='-', color='k', linewidth=1, alpha=1.0, label='filtered')
    current_axes.set_title('Heading (Degrees)', fontsize=10)
    current_axes.set_ylabel('Signals', fontsize=10)
    if y1lim is not None:
        current_axes.set_ylim(y1lim)
    current_axes.legend(loc='upper left')
    current_axes.grid(which='both',axis='both')

    current_axes = axes[1]
    current_axes.plot(raw_data.roll, linestyle='-', color='b', linewidth=4, alpha=0.5, label='raw')
    if do_filtered_data:
        current_axes.plot(filtered_data.roll, linestyle='-', color='k', linewidth=1, alpha=1.0, label='filtered')
    current_axes.set_title('Roll (Degrees)', fontsize=10)
    current_axes.set_ylabel('Signals', fontsize=10)
    if y2lim is not None:
        current_axes.set_ylim(y2lim)
    current_axes.legend(loc='upper left')
    current_axes.grid(which='both',axis='both')

    current_axes = axes[2]
    current_axes.plot(raw_data.pitch, linestyle='-', color='b', linewidth=4, alpha=0.5, label='raw')
    if do_filtered_data:
        current_axes.plot(filtered_data.pitch, linestyle='-', color='k', linewidth=1, alpha=1.0, label='filtered')
    current_axes.set_title('Pitch (Degrees)', fontsize=10)
    current_axes.set_ylabel('Signals', fontsize=10)
    if y3lim is not None:
        current_axes.set_ylim(y3lim)
    current_axes.legend(loc='upper left')
    current_axes.grid(which='both',axis='both')

In [8]:
import pandas as pd
def make_prepared_data_df(test_data,frame_ind=0):
    prepared_data = pd.DataFrame({
        't':test_data['t_data'][frame_ind],
        'x':test_data['x_data'][frame_ind],
        'y':test_data['y_data'][frame_ind],
        'z':test_data['z_data'][frame_ind]},
        index=range(len(test_data['t_data'][frame_ind])))
    if 'qw_data' in test_data.keys():
        prepared_data = prepared_data.join(
            pd.DataFrame({
                'qw':test_data['qw_data'][frame_ind],
                'qx':test_data['qx_data'][frame_ind],
                'qy':test_data['qy_data'][frame_ind],
                'qz':test_data['qz_data'][frame_ind]},
                index=range(len(test_data['t_data'][frame_ind]))),
            how='inner')
    if 'heading_data' in test_data.keys():
        prepared_data = prepared_data.join(
            pd.DataFrame({
                'heading':test_data['heading_data'][frame_ind],
                'roll':test_data['roll_data'][frame_ind],
                'pitch':test_data['pitch_data'][frame_ind]},
                index=range(len(test_data['t_data'][frame_ind]))),
            how='inner')
    return prepared_data

def log_test_data_to_file(test_data,data_file_base_name):
    # Repackage test data for writing to csv via Pandas
    prepared_data = make_prepared_data_df(test_data,frame_ind=0)
    # - Write data to file
    log_time_str = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
    prepared_data.to_csv(
        '{base_str}_{time_str}.csv'.format(base_str=data_file_base_name,time_str=log_time_str),
        index=False,
        encoding='utf-8')

In [171]:
p.is_alive()

False

In [110]:
p.join()